In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
'''
import tensorflow as tf
pre_model= tf.keras.applications.VGG16(include_top=False,
                             weights='imagenet',
                             input_shape=(150,150,3))
                             
                             '''

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:

train_datagen = ImageDataGenerator(rescale = 1./255,rotation_range=15,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    shear_range=0.15,
                                    zoom_range=0.15,
                                    horizontal_flip=True,
                                  )
val_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
#Batch Size and Image Size
batch_sz = 64
sz = 48

train_generator = train_datagen.flow_from_directory('../input/fer-dsc-v3/final_data_v5/final_data_v5/Train',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

val_generator = val_datagen.flow_from_directory('../input/fer-dsc-v3/final_data_v5/final_data_v5/Test',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

Found 13651 images belonging to 4 classes.
Found 1649 images belonging to 4 classes.


In [5]:
from keras.applications.resnet50 import ResNet50

pre_model = ResNet50(include_top = False,
                 weights = 'imagenet',
                 input_shape = (48,48,3))

for layer in pre_model.layers:
    layer.trainable=False

last_output = pre_model.output

94773248/94765736 [==============================] - 1s 0us/step


In [6]:
pre_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 24, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
for layer in pre_model.layers:
    layer.trainable=False

last_output = pre_model.get_layer('conv2_block3_2_conv').output

In [8]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(128,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4,activation='softmax')(x)

model = tf.keras.Model(pre_model.input, x)

In [9]:
opt = tf.keras.optimizers.RMSprop(learning_rate=1e-5)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_generator,
                   validation_data=val_generator,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                   validation_steps=val_generator.samples//val_generator.batch_size,
                    epochs=30
                    )

Epoch 1/30
213/213 [==============================] - 46s 215ms/step - loss: 1.4049 - accuracy: 0.2601 - val_loss: 1.3858 - val_accuracy: 0.2775
Epoch 2/30
213/213 [==============================] - 22s 104ms/step - loss: 1.3902 - accuracy: 0.2573 - val_loss: 1.3839 - val_accuracy: 0.2675
Epoch 3/30
213/213 [==============================] - 22s 103ms/step - loss: 1.3865 - accuracy: 0.2667 - val_loss: 1.3788 - val_accuracy: 0.2750
Epoch 4/30
213/213 [==============================] - 23s 106ms/step - loss: 1.3864 - accuracy: 0.2621 - val_loss: 1.3786 - val_accuracy: 0.2788
Epoch 5/30
213/213 [==============================] - 22s 103ms/step - loss: 1.3855 - accuracy: 0.2664 - val_loss: 1.3779 - val_accuracy: 0.2912
Epoch 6/30
213/213 [==============================] - 22s 103ms/step - loss: 1.3862 - accuracy: 0.2643 - val_loss: 1.3786 - val_accuracy: 0.2862
Epoch 7/30
213/213 [==============================] - 22s 104ms/step - loss: 1.3846 - accuracy: 0.2648 - val_loss: 1.3752 - val_ac

In [10]:
for layer in model.layers:
    layer.trainable = True

In [11]:
opt1 = tf.keras.optimizers.Adam(learning_rate = 1e-4)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1 = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = 100)
                    #callbacks = [callbacks])

Epoch 1/100
214/214 [==============================] - 24s 112ms/step - loss: 1.3889 - accuracy: 0.3253 - val_loss: 1.4611 - val_accuracy: 0.2969
Epoch 2/100
214/214 [==============================] - 22s 105ms/step - loss: 1.2824 - accuracy: 0.3937 - val_loss: 1.7915 - val_accuracy: 0.2625
Epoch 3/100
214/214 [==============================] - 24s 111ms/step - loss: 1.2119 - accuracy: 0.4442 - val_loss: 1.2915 - val_accuracy: 0.3569
Epoch 4/100
214/214 [==============================] - 24s 110ms/step - loss: 1.1397 - accuracy: 0.4833 - val_loss: 1.1475 - val_accuracy: 0.5038
Epoch 5/100
214/214 [==============================] - 22s 105ms/step - loss: 1.1024 - accuracy: 0.5137 - val_loss: 1.0623 - val_accuracy: 0.5475
Epoch 6/100
214/214 [==============================] - 23s 109ms/step - loss: 1.0546 - accuracy: 0.5366 - val_loss: 1.0346 - val_accuracy: 0.5663
Epoch 7/100
214/214 [==============================] - 23s 106ms/step - loss: 1.0245 - accuracy: 0.5509 - val_loss: 0.9483 -

Epoch 57/100
214/214 [==============================] - 25s 117ms/step - loss: 0.6783 - accuracy: 0.7261 - val_loss: 0.7953 - val_accuracy: 0.6744
Epoch 58/100
214/214 [==============================] - 24s 111ms/step - loss: 0.6792 - accuracy: 0.7243 - val_loss: 0.7217 - val_accuracy: 0.7100
Epoch 59/100
214/214 [==============================] - 24s 113ms/step - loss: 0.6778 - accuracy: 0.7290 - val_loss: 0.7466 - val_accuracy: 0.7094
Epoch 60/100
214/214 [==============================] - 24s 112ms/step - loss: 0.6697 - accuracy: 0.7309 - val_loss: 0.7531 - val_accuracy: 0.6950
Epoch 61/100
214/214 [==============================] - 24s 111ms/step - loss: 0.6609 - accuracy: 0.7337 - val_loss: 0.7271 - val_accuracy: 0.6975
Epoch 62/100
214/214 [==============================] - 24s 112ms/step - loss: 0.6585 - accuracy: 0.7336 - val_loss: 0.7481 - val_accuracy: 0.7025
Epoch 63/100
214/214 [==============================] - 23s 109ms/step - loss: 0.6600 - accuracy: 0.7260 - val_loss: 0

In [ ]:
opt1 = tf.keras.optimizers.Adam(learning_rate = 1e-4)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1 = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = 50)
                    #callbacks = [callbacks])

In [12]:

model.save('resnet50.h5')

model.save_weights('resnet50_weights.h5')


In [13]:

test = '../input/fer-dsc-v3/final_data_v5/final_data_v5/Test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory (test, batch_size=256, class_mode='categorical', target_size=(48,48))

model.evaluate(test_generator, steps=4)

Found 1649 images belonging to 4 classes.
4/4 [==============================] - 1s 139ms/step - loss: 0.7108 - accuracy: 0.7207


[0.710780918598175, 0.720703125]